# **Rebalancing Tool - SW** 
    
This tool is used to validate if funds on SW platform are within predefined tolerance ranges.

In [1]:
import pandas as pd

from utils.glidepath import merged_glidepaths
import utils.pipeline as pl
import utils.validations as val
import utils.config as config


## Settings


In [2]:
PROVIDER = 'SW'

# File paths
sw_weekly = 'data/sw_weekly'
sw_reference = pd.read_csv('data/reference/sw_reference.csv')
sw_static_funds_targets = pd.read_csv('data/reference/sw_static_funds_targets.csv')
sw_output = 'data/output/sw_output.csv'

# Tolerance levels
MIN_RANGE = -0.03
MAX_RANGE = 0.03

glidepath_lookback = 'YES'

webhook_url = config.webhook_url


## Processing

### Preprocess data

In [3]:

all_glidepaths = merged_glidepaths()
df = pl.load_and_preprocess_data_sw(sw_weekly, sw_reference)
print(df.info())    


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   fund_label       199 non-null    object        
 1   fund_underlying  199 non-null    object        
 2   fund_key         199 non-null    object        
 3   actual_weight    199 non-null    float64       
 4   target_weight    199 non-null    float64       
 5   valuation        199 non-null    float64       
 6   date             199 non-null    datetime64[ns]
 7   fund_glidepath   130 non-null    object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 12.6+ KB
None


### Test

In [4]:
val.validate_columns(df, pl.pre_columns)
val.test_no_duplicates(df)  


Validation Test: Column date has been validated
Validation Test: Column fund_label has been validated
Validation Test: Column fund_underlying has been validated
Validation Test: Column fund_key has been validated
Validation Test: Column fund_glidepath has been validated
Validation Test: Column valuation has been validated
Validation Test: Column actual_weight has been validated
Validation Test: Column target_weight has been validated
Validation Test: No duplicates found



### Main processing and calculations

In [5]:
df = pl.add_glidepath_data(df, glidepath_lookback)
df = pl.add_lookup_values(df, all_glidepaths)
df = pl.add_static_target_values(df, sw_static_funds_targets)
df = pl.calculate_difference_final(df)

df.to_csv(sw_output, index=False)
print('Data processed and saved to data/output/sw_output.csv')
print('')
print(df.info())
print('')

Lookup values added successfully and count test is passed.

Validation Test: No negative values found in the column "target_val"

Data processed and saved to data/output/sw_output.csv

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   fund_label                  199 non-null    object        
 1   fund_underlying             199 non-null    object        
 2   fund_key                    199 non-null    object        
 3   actual_weight               199 non-null    float64       
 4   target_weight               199 non-null    float64       
 5   valuation                   199 non-null    float64       
 6   date                        199 non-null    datetime64[ns]
 7   fund_glidepath              130 non-null    object        
 8   glidepath                   199 non-null    object        
 9   y

### Adding mask providing final output

In [6]:
validated_df, date_result = pl.check_range(df, MIN_RANGE, MAX_RANGE)
validated_df = pl.post_transformations(validated_df)


Post-transformations completed successfully.


### Test

In [7]:
test = val.test_no_invalid_dates(validated_df)


Validation Test: Dates are in range 


## Send Message

In [8]:
#pl.send_dataframe_to_teams(validated_df, webhook_url, provider=PROVIDER)
